In [ ]:
# init 
#train_loss = [0,0,0]
#val_loss = [0,0,0]
index = 0
train_index_array = np.arange(x_train.shape[0])
#val_index_array = np.arange(x_val.shape[0])

# plots
history_train_losses = [[],[],[]]
#history_val_losses = [[],[],[]]
history_train_acc=[]
#history_val_acc=[]
iterazioni=[]

In [ ]:
import metrics

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _ = model.predict(x_train, verbose=0)
        #qval, _ = model.predict(x_val, verbose=0)
        p = target_distribution(q) # update the auxiliary target distribution p
        #pval = target_distribution(qval)
        #import pdb; pdb.set_trace()

        # evaluate the clustering performance
        y_train_pred = q.argmax(1)
        #y_val_pred = qval.argmax(1)
        if y_train is not None:
            train_acc = np.round(metrics.acc(y_train, y_train_pred), 5)
            train_nmi = np.round(metrics.nmi(y_train, y_train_pred), 5)
            train_ari = np.round(metrics.ari(y_train, y_train_pred), 5)
            train_loss = np.round(train_loss, 5)
            print('Iter', ite, ': Acc tr', train_acc, ', nmi tr', train_nmi, ', ari tr', train_ari, '; loss tr=', train_loss)
        #if y_val is not None:
        #    val_acc = np.round(metrics.acc(y_val, y_val_pred), 5)
        #    val_nmi = np.round(metrics.nmi(y_val, y_val_pred), 5)
        #    val_ari = np.round(metrics.ari(y_val, y_val_pred), 5)
        #    val_loss = np.round(val_loss, 5)
        #    print('Iter', ite, ': Acc val', val_acc, ', nmi tr', val_nmi, ', ari tr', val_ari, '; val_loss=', val_loss) 
            
        # check stop criterion
        delta_label = np.sum(y_train_pred != y_pred_last).astype(np.float32) / y_train_pred.shape[0]
        y_pred_last = np.copy(y_train_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    
    # train on batch
    idx_train = train_index_array[index * batch_size: min((index+1) * batch_size, x_train.shape[0])]

    train_loss = model.train_on_batch(x=x_train[idx_train], 
                                      y=[p[idx_train], x_train[idx_train]])
    history_train_losses[0].append(train_loss[0])
    history_train_losses[1].append(train_loss[1])
    history_train_losses[2].append(train_loss[2])
    history_acc.append(train_acc)

    #idx_val = val_index_array[index * batch_size: min((index+1) * batch_size, x_val.shape[0])]
    #val_loss = model.test_on_batch(x=x_val[idx_val], 
    #                               y=[pval[idx_val], x_val[idx_val]])
    #history_val_losses[0].append(val_loss[0])
    #history_val_losses[1].append(val_loss[1])
    #history_val_losses[2].append(val_loss[2])
    #history_acc.append(val_acc)
    
    index = index + 1 if (index + 1) * batch_size <= x_train.shape[0] else 0
    
    iterazione.append(ite)
    
    if ite % save_interval == 0:
        # save DCEC model checkpoints
        print('saving model to:', save_dir + '/dcec_model_' + str(ite) + '.h5')
        model.save_weights(save_dir + '/dcec_model_' + str(ite) + '.h5')

# save the trained model
print('saving model to:', save_dir + '/dcec_model_final.h5')
model.save_weights(save_dir + '/dcec_model_final.h5')

In [ ]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    input_img = Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

In [ ]:
#use this function for reshaping arrays for DenseAutoEncoder
def ReshapeDAE(array):
    array = array.reshape(array.shape[0], -1)
    array = array/255.
    return array

#scegli la funzione
f = ReshapeDAE

x_train = f(x_train)
x_val = f(x_val)
x_test = f(x_test)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

dims = [x.shape[-1], 500, 500, 2000, 3]
print(dims[0])

# THIS IS THE AUTOENCODER CON IL DENSE PRIMA DELL'EMBEDDED E DOPO L'EMBEDDED: il train ha data come risultati delle immagini schifo (vedi risultati_autoencoder_schifo.png)



def autoencoderConv2D_1(input_shape=(128, 128, 1), filters=[32, 64, 128, 256, 3]):
    input_img = Input(shape=input_shape)
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)

    x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2')(x)

    x = Conv2D(filters[2], 3, strides=2, padding=pad3, activation='relu', name='conv3')(x)

    x = Flatten()(x)
    
    x = Dense(units=filters[3])(x) ##
    
    encoded = Dense(units=filters[4], name='embedding')(x)
    
    x = Dense(units=filters[3], activation='relu')(encoded)
    
    x = Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu')(x) ##

    x = Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2]))(x)
    
    x = Conv2DTranspose(filters[1], 3, strides=2, padding=pad3, activation='relu', name='deconv3')(x)

    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1')(x)
    
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

In [3]:
from keras.models import Model
from keras import backend as K
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model

Using TensorFlow backend.


In [14]:
def autoencoderConv2D_1(input_shape=(128, 128, 1), filters=[32, 64, 128, 256, 3]):
    input_img = Input(shape=input_shape)
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)

    x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2')(x)

    x = Conv2D(filters[2], 3, strides=2, padding=pad3, activation='relu', name='conv3')(x)

    x = Flatten()(x)
    
    encoded = Dense(units=filters[3], name='embedding')(x)
    
    y = Dense(units=filters[4])(encoded)
    
    x = Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu')(encoded)

    x = Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2]))(x)
    
    x = Conv2DTranspose(filters[1], 3, strides=2, padding=pad3, activation='relu', name='deconv3')(x)

    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1')(x)
    
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=y, name='encoder')

In [18]:
autoencoder, encoder = autoencoderConv2D_1()

print('ENCODER')
encoder.summary()

print('AUTOENCODER')
autoencoder.summary()

ENCODER
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        832       
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 32, 64)        51264     
_________________________________________________________________
conv3 (Conv2D)               (None, 16, 16, 128)       73856     
_________________________________________________________________
flatten_9 (Flatten)          (None, 32768)             0         
_________________________________________________________________
embedding (Dense)            (None, 256)               8388864   
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 771       
To

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Category10_10
from bokeh.resources import INLINE
import bokeh.io
bokeh.io.output_notebook(INLINE) 
from io import BytesIO
from PIL import Image
import base64
def embeddable_image(data):
    data = data[:,:,0]
    buffer = BytesIO()
    plt.imsave(buffer, data, cmap='gray')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

import pandas as pd

emb = model.predict(x_test)[0]
digits_df = pd.DataFrame(emb, columns=('x', 'y','z'))
digits_df['labels'] = [str(i) for i in y_test]
digits_df['image'] = list(map(embeddable_image, list(x_test)))
#digits_df['image_name'] = [dset_train.x_name[i] for i in range(len(dset_train))]
digits_df = digits_df
datasource = ColumnDataSource(digits_df)
color_mapping = CategoricalColorMapper(factors=['0', '1', '2'],
                                       palette=Category10_10)
plot_figure = figure(
    plot_width=800,
    plot_height=800,
    tools=('pan, wheel_zoom, reset')
)
plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 1px 1px 1px 1px'/>
    </div>
    <div>
        <span style='font-size: 13px'>@labels</span>
    </div>
</div>
"""))
plot_figure.circle(
    'x',
    'y',
    source=ColumnDataSource(digits_df),
    color=dict(field='labels', transform=color_mapping)
    )
plot_figure.cross(
    x=centers[:,0],
    y=centers[:,1],
    size=20,
    line_width=2
)

#plot_figure.legend.location = "top_left"
#plot_figure.legend.click_policy="hide"
show(plot_figure)